In [35]:
import math
import os
import sys
import time

import glob
from xlrd import open_workbook
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score
import dask.dataframe as dd
import dask.array as da
from util.calc_util import split_train_and_test_data
from util.cfg import config
from util.file_util import mkdirs_if_not_exist, out_result, prepare_scutfbp5500
from util.vgg_face_feature import extract_feature
import random

from keras.engine import  Model
from keras.layers import Input
from keras_vggface.vggface import VGGFace
from keras.preprocessing import image
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
vgg_model = VGGFace() # pooling: None, avg or max

def extract_feature_tf2(file, layer_name):
   
    out = vgg_model.get_layer(layer_name).output
    vgg_model_new = Model(vgg_model.input, out)
    img = image.load_img(file, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=1) # or version=2
    features = vgg_model_new.predict(x)
    return features.ravel()


def ecust_train_and_test_set():
    """
    ECUST-FBP dataset creation
    """
    
    DATA_DIR = '/home/ubuntu/mtcnn_aligned_images/female/'
    LABELS_FILE = '/home/ubuntu/ECUST_FBP/scores/public_generic_all.xlsx'
    
    
    df = pd.read_excel(LABELS_FILE)  
    labels_dict= {}
    for root, dirs, files in os.walk(DATA_DIR):
     
      for file in files: 
        
        f = os.path.join(root, file)
        df1 = df.loc[df['image']==file]
        if (len(df1.index)==1):
         
           labels_dict[f] = float(df1['mean'])
  
      keys =  list(labels_dict.keys())
      random.shuffle(keys)
    d1 = dict(list(labels_dict.items())[len(labels_dict)//2:])
    d2 = dict(list(labels_dict.items())[:len(labels_dict)//2])
    return d1, d2      

train,test = ecust_train_and_test_set()


/home/ubuntu/mtcnn_aligned_images/female/midestern/abi-lewis-5f8i3ckEswk-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/56.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/pexels-ricael-sousa-1837158.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/3.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/kirill-balobanov-Ef3AOkt6hHg-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/girl-1360874_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/houcine-ncib-2Qg42qfZpRo-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/s-jDpjN4U5mxE-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/warrior-1240995_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/joao-paulo-de-souza-oliveira-7-CSeYLI-KM-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/ashkan-forouzani-SNYaOnl22u8-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/jan-kopriva-VKHv4GLr5O0-unsplash.jpg
/home/ubuntu/mtcnn_al

/home/ubuntu/mtcnn_aligned_images/female/asian/ahmad-qime-in2aoHwX5bA-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/alex-sorto-suymY_8bk4Y-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/pexels-min-an-1586654.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/john-mark-ramos-SGINHNM-vhk-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/pexels-mental-health-america-(mha)-5531323.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/altanbagana-jargal-o79BpJ-tgew-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/afif-kusuma-y-mT2OATY6k-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/dani-bayuni-FhuHS2959uI-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/portrait-3622823_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/woman-2294787_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/vietnamese-4646223_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/woman-3951558_1920.jpg
/home/ubuntu/mtcnn_aligned_image

/home/ubuntu/mtcnn_aligned_images/female/black/engin-akyurt-Obi2AUzEZyU-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/black/wadi-lissa-PnCTTH_mmYg-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/black/noah-buscher-AR7aumwKr2s-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/black/cassandra-hamer-t2obqqft_xM-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/black/pexels-rodnae-productions-4921034.jpg
/home/ubuntu/mtcnn_aligned_images/female/black/suad-kamardeen-4VN2WWVWqDs-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/black/woman-5932291_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/black/jessica-felicio-8vgkMCSq4EU-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/black/steward-masweneng-Ws4YEqBafus-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/black/model-4676037_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/black/brenna-huff-0bayIkan-Ao-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/black/39.jpg
/home/ubuntu/mtcnn_aligned_imag

/home/ubuntu/mtcnn_aligned_images/female/hispanic/obi-onyeador-7ckRPDhlgOk-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/hispanic/khaled-ghareeb-bcQh4nVETaY-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/hispanic/tabitha-turner-NlN-dY9Fk3A-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/hispanic/mateus-campos-felipe-64sktAOWasI-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/hispanic/ayo-ogunseinde-THIs-cpyebg-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/hispanic/bruno-dias-D22Tgx6TP5M-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/hispanic/beautiful-1869534_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/hispanic/madison-hedgecock-tacwAxr3PvU-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/hispanic/alex-blajan-iRsktlI-hls-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/hispanic/woman-5225288_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/hispanic/woman-2773007_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/hispanic/matteo-

/home/ubuntu/mtcnn_aligned_images/female/caucasian/winter-1520232_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/white-cat-with-blonde-4655233_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/kareya-saleh-tLKOj6cNwe0-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/jasmine-b-qaSPhHr_Iug-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/fezbot2000-V4CU3OBsrEk-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/laura-dewilde-dhCxlHodZvM-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/beautiful-young-woman-635271_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/emmanuel-bior-N06QdiZgK_E-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/ratiu-bia-7fzINre0NQE-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/pexels-skyler-ewing-4680663.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/girl-1471048_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/girl-233488

/home/ubuntu/mtcnn_aligned_images/female/caucasian/joy-4196371_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/fashion-1284308_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/hichem-dahmani-EhcwOfAn-mA-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/irina-iriser-T9_NHo5d0pc-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/apostolos-vamvouras-egW3HtJfsrE-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/people-4919068_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/sharon-mccutcheon-EpsQ8oVcDj4-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/elijah-m-henderson-r9lTXKNtq7I-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/ann-agterberg-WqATKbXqGZQ-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/karsten-winegeart-s2MjR3xoJkE-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/mike-austin-t24Wz_EFerc-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female

/home/ubuntu/mtcnn_aligned_images/female/caucasian/girl-2772177_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/jj-jordan-t9tNNBEPwV0-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/clayton-cardinalli-ZqmmwcE1DQ8-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/grimace-1012862_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/kayla-maurais-LLMLMR_Dwkg-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/megan-bagshaw-VqblHgABSVA-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/shilja-74758_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/noah-buscher-AUM5vcnuYd4-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/gemma-chua-tran-u1eZM06-MPw-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/omid-armin-VS4Bg3tWWcI-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/woman-3073289_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/jen-theodore-7A4TvZEW8zY

/home/ubuntu/mtcnn_aligned_images/female/caucasian/alexander-scott-lambley-i2AmfpRO0Cc-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/portrait-3292287_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/redheads-4606477_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/freestocks-9UVmlIb0wJU-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/brooke-cagle-BwhJOU0sgdY-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/actress-1893312_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/beauty-1132617_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/girl-3576679_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/girl-4283996_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/portrait-886039_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/tanya-trofymchuk-gCKYhaa4b2E-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/girl-4185856_1920.jpg
/home/ubuntu/mtcnn_aligned_imag

/home/ubuntu/mtcnn_aligned_images/female/indian/pexels-rafi-ahmed-haven-1253364.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/christina-wocintechchat-com-z5SOVx-y3F4-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/jewellery-4255833_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/60.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/beauty-4175607_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/pexels-mabel-lee-2739792.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/bride-5170727_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/portrait-4773342_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/juniar-baresi-1g-BQ3wkgWI-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/zhang-kaiyv-voWjZ-NIrqo-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/33.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/dollar-gill-Ciujwy5epq4-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/ethan-johnson-n815BbX98ds-unspl

In [37]:

# Layer Features

    

    train_vec = list()
    train_label = list()
    test_vec = list()
    test_label = list()

    for k, v in train.items():
        feature = np.concatenate((extract_feature_tf2(k, layer_name="conv5_2"), extract_feature_tf2(k, layer_name="conv5_3")),
                                 axis=0)
      
        train_vec.append(feature)
        train_label.append(v)

    for k, v in test.items():
        feature = np.concatenate((extract_feature_tf2(k, layer_name="conv5_2"), extract_feature_tf2(k, layer_name="conv5_3")),
                                 axis=0)
        test_vec.append(feature)
        test_label.append(v)
 
    pca = PCA(n_components=600)
    train_vec = pca.fit_transform(train_vec)
    test_vec = pca.fit_transform(test_vec)    
    reg = linear_model.BayesianRidge()
    #print(train_label)
    train_vec = np.array(train_vec)
    train_label = np.array(train_label)
    reg.fit(train_vec, train_label)
   #mkdirs_if_not_exist('./model')
   #joblib.dump(reg, config['eccv_fbp_reg_model'])

    predicted_label = reg.predict(np.array(test_vec))
    mae_lr = round(mean_absolute_error(np.array(test_label), predicted_label), 4)
    rmse_lr = round(math.sqrt(mean_squared_error(np.array(test_label), predicted_label)), 4)
    pc = round(np.corrcoef(test_label, predicted_label)[0, 1], 4)

    print('===============The Mean Absolute Error of Model is {0}===================='.format(mae_lr))
    print('===============The Root Mean Square Error of Model is {0}===================='.format(rmse_lr))
    print('===============The Pearson Correlation of Model is {0}===================='.format(pc))

    csv_tag = time.time()

    mkdirs_if_not_exist('./result')
    df = pd.DataFrame([mae_lr, rmse_lr, pc])
    df.to_csv('./result/performance_%s.csv' % csv_tag, index=False)

    out_result(list(test.keys()), predicted_label.flatten().tolist(), test_label, None,
               path='./result/detail_%s.csv' % csv_tag)

===============The Mean Absolute Error of Model is 1.2657====================
===============The Root Mean Square Error of Model is 1.5823====================
===============The Pearson Correlation of Model is -0.1563====================


In [2]:
import ctypes
print(ctypes.sizeof(ctypes.c_voidp))

8
